In [1]:
import sys
sys.path.extend(['..','../..'])
from sqlalchemy import Column, Integer, VARCHAR, TIMESTAMP, NUMERIC
from sqlalchemy import create_engine
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker
import logging
import datetime
from config import conn_string
import os
from db.models import TableUsers, TableUserAirports, TableAirportCodes, DataAccessLayer
import pandas as pd
# from dotenv import load_dotenv
#
# load_dotenv()
#
# DATABASE_USER = str(os.getenv("DBUSER"))
# DATABASE_PASS = str(os.getenv("DBPASSWORD"))
# DATABASE_NAME = str(os.getenv("DBNAME"))
# DATABASE_PORT = str(os.getenv("DBPORT"))
# DATABASE_HOST = str(os.getenv("DBHOST"))
#
# conn_string = f'postgresql://'
# conn_string += f'{DATABASE_USER}:{DATABASE_PASS}'
# conn_string += f'@{DATABASE_HOST}:{DATABASE_PORT}/{DATABASE_NAME}'


In [6]:
fligts_pth= os.path.join('..','FlightDeals.xlsx')
flightsdf = pd.read_excel(fligts_pth, engine='openpyxl')

In [7]:
airportsdf_pth = os.path.join('..','world-airport-codes.xlsx')
airportsdf = pd.read_excel(airportsdf_pth,engine='openpyxl')

In [8]:
airportsdf.head()

,AirportName,Country,IATA,Latitude,Longitude
0,Anaa Airport,French Polynesia,AAA,-17.352600,-145.509995
1,Arrabury Airport,Australia,AAB,-26.700001,141.050003
2,El Arish International Airport,"El Arish, Egypt",AAC,31.073299,33.835800
3,Ad-Dabbah Airport,"Ad-Dabbah, Sudan",AAD,17.592778,33.959167
4,Annaba Airport,"Annabah, Algeria",AAE,36.822201,7.809170


In [5]:
me = TableUsers('224483345', "Alex")


In [2]:
dal = DataAccessLayer(conn_string)

In [3]:
session = dal.get_session()

In [8]:
# session.add(me)
# session.commit()

In [11]:
airportslist = []
for row in airportsdf.iterrows():
    ap= TableAirportCodes(row[1][0], row[1][2], row[1][1])
    airportslist.append(ap)

In [4]:
from tqdm.auto import tqdm

In [15]:
# for row in tqdm(airportsdf.iterrows()):
#     ap= TableAirportCodes(row[1][0], row[1][2], row[1][1])
#     session.add(ap)
#     session.commit()

In [13]:
airportsdf['airport_country'] = airportsdf['AirportName'] + ', ' + airportsdf['Country']

In [15]:
# session.rollback()
userid = '224483345'
def get_destinations(session, user_id):
    destinations_dict = dict()
    for el in session.query(TableUserAirports).filter_by(user_id=user_id).all():
        destinations_dict[el.airport_code] = int(el.price)
    return destinations_dict

In [16]:
get_destinations(session, userid)

{'AAA': 4000, 'LED': 2300, 'SWO': 2400, 'SVO': 4999, 'DME': 3000}

In [18]:
for el in session.query(TableUserAirports).filter_by(user_id='224483345'):
    print(el)

InvalidRequestError: This Session's transaction has been rolled back due to a previous exception during flush. To begin a new transaction with this Session, first issue Session.rollback(). Original exception was:  (Background on this error at: http://sqlalche.me/e/13/7s2a)

In [15]:
iata_name_dict = dict(zip(airportsdf['IATA'].tolist(), airportsdf['airport_country'].tolist()))

In [16]:
import pickle

In [17]:
with open('iata_name_dict.pkl', 'wb') as f:
    pickle.dump(iata_name_dict, f)

In [ ]:
session.bulk_save_objects(airportslist)
session.commit()

In [ ]:
TableUserAirports